# Сервис для чтения книг

**Цель исследования:**
Проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.
С помощью SQL-запросов решить задания.

**Задания**

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Этапы исследования:**

- Доступ к базе данных;
- Исследование данных;
- Решение задач;
- Выводы.

**Описание данных:**

Таблица *books*

Содержит данные о книгах:

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица *authors*

Содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.

Таблица *publishers*

Содержит данные об издательствах:

- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица *ratings*

Содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица *reviews*

Содержит данные о пользовательских обзорах:

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

In [1]:
# Импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

## Доступ к базе данных

In [2]:
# Устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# Сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

# Чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

## Исследование данных

In [9]:
# Посмотрим на данные в таблицах
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']
for table_name in tables:
    query = f"SELECT * FROM {table_name} LIMIT 5"
    display(get_sql_data(query))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Видим что у нас есть пять таблиц с данными о книгах, их авторах, издательствах, проставленными оценками(или рейтингом) и отзывами.

## Решение задач

### Посчитайте, сколько книг вышло после 1 января 2000 года;

In [10]:
# Сформируем запрос и выведе данные
query = '''SELECT count(book_id)
           FROM books
           WHERE publication_date > '2000-01-01' '''
get_sql_data(query)

,count
0,819


В результате получаем что после 1 января 2000 года вышла 819 книг.

### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [11]:
query = '''With amreviews as (SELECT book_id,
                                     count(username) as amount_reviews
                              FROM reviews
                              GROUP BY book_id),
           avrating as (SELECT book_id,
                               ROUND(AVG(rating),2) as avg_ratings
                        FROM ratings
                        GROUP BY book_id)
           SELECT b.title,
                  am.amount_reviews,
                  av.avg_ratings
           FROM books as b
           LEFT JOIN amreviews as am ON b.book_id = am.book_id
           LEFT JOIN avrating as av ON b.book_id = av.book_id'''
get_sql_data(query)

,title,amount_reviews,avg_ratings
0,The Body in the Library (Miss Marple #3),2.0,4.50
1,Galápagos,2.0,4.50
2,A Tree Grows in Brooklyn,5.0,4.25
3,Undaunted Courage: The Pioneering First Missio...,2.0,4.00
4,The Prophet,4.0,4.29
...,...,...,...
995,Alice in Wonderland,4.0,4.23
996,A Woman of Substance (Emma Harte Saga #1),2.0,5.00
997,Christine,3.0,3.43
998,The Magicians' Guild (Black Magician Trilogy #1),2.0,3.50


Для подсчета использовали две отдельные таблицы, после чего с помощью джойна объединили оценки и обзоры со списком книг. И получили таблицу с количеством обзоров и средним рейтингом для всех книг.

In [12]:
query = '''With amreviews as (SELECT book_id,
                                     count(username) as amount_reviews
                              FROM reviews
                              GROUP BY book_id),
           avrating as (SELECT book_id,
                               ROUND(AVG(rating),2) as avg_ratings
                        FROM ratings
                        GROUP BY book_id)
           SELECT b.title,
                  am.amount_reviews,
                  av.avg_ratings
           FROM books as b
           LEFT JOIN amreviews as am ON b.book_id = am.book_id
           LEFT JOIN avrating as av ON b.book_id = av.book_id
           WHERE av.avg_ratings = 5'''
get_sql_data(query)

,title,amount_reviews,avg_ratings
0,Light in August,2.0,5.0
1,Act of Treason (Mitch Rapp #9),2.0,5.0
2,The Demon-Haunted World: Science as a Candle i...,2.0,5.0
3,Dead Souls,2.0,5.0
4,March,2.0,5.0
5,Wherever You Go There You Are: Mindfulness Me...,2.0,5.0
6,Neil Gaiman's Neverwhere,2.0,5.0
7,Evening Class,2.0,5.0
8,The Ghost Map: The Story of London's Most Terr...,2.0,5.0
9,Welcome to Temptation (Dempseys #1),2.0,5.0


При этом если выбрать рейтинг равный 5, получим что у нас таких целых 43 книги.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [13]:
query = '''With publish as (SELECT b.book_id,
                                   p.publisher
                            FROM books as b
                            JOIN publishers as p ON b.publisher_id = p.publisher_id
                            WHERE b.num_pages > 50)
           SELECT publisher,
                  count(book_id) as amount_books
           FROM publish
           GROUP BY publisher
           ORDER BY count(book_id) DESC
           LIMIT 1'''
get_sql_data(query)

,publisher,amount_books
0,Penguin Books,42


Для решения задачи объединяем таблицы книг и издательств, сразу убираем книги менее 50 страниц. После чего считаем для каждого издательства количество книг. Сортируем по убыванию и выводим первое издаетльствои из списка. Получаем что искомое издательство - Penguin Books. Издали они 42 книги.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [14]:
query = '''WITH book_ratings AS (SELECT r.book_id,
                                        AVG(r.rating) AS avg_rating,
                                        COUNT(r.rating) AS total_ratings
                                 FROM ratings r
                                 GROUP BY r.book_id
                                 HAVING COUNT(r.rating) >= 50),
           author_average_ratings AS (SELECT a.author_id,
                                             SUM(br.total_ratings) AS total_ratings,
                                             AVG(br.avg_rating) AS avg_rating
                                      FROM authors a
                                      JOIN books b ON a.author_id = b.author_id
                                      JOIN book_ratings br ON b.book_id = br.book_id
                                      GROUP BY a.author_id)
           SELECT a.author,
                  aar.total_ratings,
                  aar.avg_rating
           FROM authors a
           JOIN author_average_ratings aar ON a.author_id = aar.author_id
           ORDER BY aar.avg_rating DESC
           LIMIT 1'''
get_sql_data(query)

,author,total_ratings,avg_rating
0,J.K. Rowling/Mary GrandPré,310.0,4.283844


В результате получили что самую высокую оценку имеет J.K. Rowling/Mary GrandPré. Оценка 4,28, при этом количество оценок 310.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [15]:
query = '''WITH user_ratings AS (SELECT username,
                                        COUNT(rating_id) AS ratings_count
                                 FROM ratings
                                 GROUP BY username
                                 HAVING COUNT(rating_id) > 48),
           user_reviews AS (SELECT username,
                                   COUNT(review_id) AS reviews_count
                            FROM reviews
                            WHERE username IN (SELECT username FROM user_ratings)
                            GROUP BY username)
           SELECT ROUND(AVG(reviews_count),2) AS average_reviews
           FROM user_reviews'''
get_sql_data(query)

,average_reviews
0,24.0


Получаем что среднее количество обзоров пользователей кто поставил более 48 оценок равно 24.

## Выводы

В результе проделанной работы получили следующие выводы:
- После 1 января 2000 года вышло 819 книг. Учитывая тот так что всего книг 1000 можно сказать что это практически 82% от всего списка.
- Для подсчета использовали две отдельные таблицы, после чего с помощью джойна объединили оценки и обзоры со списком книг. И получили таблицу с количеством обзоров и средним рейтингом для всех книг. При этом если выбрать рейтинг равный 5, получим что у нас таких целых 43 книги.
- Наибольшее число книг толще 50 страниц опубликовало издательство Penguin Books. Было издано 42 книги.
- Cамую высокую оценку имеет J.K. Rowling/Mary GrandPré. Оценка 4,28, при этом количество оценок 310.
- А среднее количество обзоров пользователей кто поставил более 48 оценок равно 24. Что говорит нам о том что пользователи кто оставляет оценки так же охотно оставляют обзоры книг.

После получения информации по задачам можно сказать о том, что пользователи продолжают читать. При этом не только оценивать книги, но и оставлять на них свои обзоры, что очень важно и помогает в подборе интересной книги. Считаю что стоит подумать в направлении того чтобы максимально поощрять пользователей делится своим мнение о прочитанных книгах. Так как на освновании имеющихся оценок и обзоров можно собрать рекомендации для других пользователей.